In [8]:
#import all necessary library
import mysql.connector
import pandas as pd
import numpy as np 
import os 
import json 


In [9]:
# --- Reusable Cleanup Function ---
def clean_and_select_dataframe(df, desired_columns=None, df_name=""):
    
    # Convert column names to lowercase first
    df.columns = df.columns.str.lower()

    # Apply specific column selection if desired_columns are provided
    if desired_columns:
        actual_desired_cols = [col for col in desired_columns if col in df.columns]
        if len(actual_desired_cols) < len(desired_columns):
            missing_cols = set(desired_columns) - set(actual_desired_cols)
            print(f"Warning: For {df_name}, some desired columns were not found: {missing_cols}")
        
        df = df.loc[:, actual_desired_cols].copy() 
    
    # Strip whitespace from all string columns
    for col in df.select_dtypes(include='object').columns:
        
        df.loc[:, col] = df[col].astype(str).str.strip()

    # Replace common empty/placeholder strings with NaN for robust empty column detection
    
    df.replace(["", "NA", "-", "N/A", "NULL", "None"], np.nan, inplace=True) 

    # Remove 'Unnamed:' columns (if any remain after explicit selection or for other DFs)
    unnamed_cols = [col for col in df.columns if 'unnamed:' in str(col).lower()]
    if unnamed_cols:
        print(f"Dropping 'Unnamed:' columns from {df_name}: {unnamed_cols}")
        df.drop(columns=unnamed_cols, inplace=True)

    # Remove columns that are entirely empty (all NaN values)
    empty_cols = [col for col in df.columns if df[col].isnull().all()]
    if empty_cols:
        print(f"Dropping entirely empty columns from {df_name}: {empty_cols}")
        df.drop(columns=empty_cols, inplace=True)

    # Drop rows where all values are NaN (if an entire row is empty after column cleanup)
    df.dropna(how="all", inplace=True)
    
    #Perform numeric conversion BEFORE filling remaining NaNs with "NA" string ---
    
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns
   
    df.loc[:, num_cols] = df[num_cols].apply(lambda col: pd.to_numeric(col, errors="coerce").fillna(0).astype(int))

    # After numeric conversions, fill any remaining NaNs (which would be in non-numeric columns)
    
    df.fillna("NA", inplace=True) 
    
    return df


In [10]:
# Define Desired Columns for Each Table (matching the updated schema)

# Common summary columns for ODI, Test
COMMON_SUMMARY_COLS = [
    "match_id", "match_format", "date", "city", "venue", "season", "gender",
    "team_1", "team_2", "toss_winner", "toss_decision", "winner",
    "win_by_runs", "win_by_wickets", "overs", "balls_per_over", "player_of_match"
]

# T20 and IPL summary columns
T20_IPL_SUMMARY_COLS = [
    "match_id", "match_format", "date", "city", "venue", "season", "gender",
    "team_1", "team_2", "toss_winner", "toss_decision", "winner",
    "win_by_runs", "win_by_wickets", "overs", "balls_per_over", "player_of_match",
    "target_runs", "target_overs", "powerplays"
]

# Test summary columns
TEST_SUMMARY_COLS = [
    "match_id", "match_format", "start_date", "end_date", "city", "venue",
    "gender", "season", "team_1", "team_2", "toss_winner", "toss_decision",
    "winner", "win_by_runs", "win_by_wickets", "balls_per_over", "player_of_match"
]

# Common ball-by-ball columns for ODI, Test
COMMON_BALL_BY_BALL_COLS = [
    "match_id", "inning_team", "over_number", "batter", "non_striker",
    "bowler", "legbyes", "wides", "no_balls", "byes", "runs_batter",
    "runs_extras", "runs_total", "wicket_kind", "player_out"
]

# Ball-by-ball columns for T20, IPL
T20_IPL_BALL_BY_BALL_COLS = [
    "match_id", "inning_team", "over_number", "batter", "non_striker",
    "bowler", "legbyes", "wides", "no_balls", "byes", "runs_batter",
    "runs_extras", "runs_total", "wicket_kind", "player_out"
]

# Player Registry columns
PLAYER_REGISTRY_COLS = [
    "match_id", "player_name", "player_id"
]


In [11]:

# Connect to TiDB Cloud
conn = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user="4DTwJ2ZR7XTapss.root",
    password="veNd2wXPKzLZwbmt",
    port=4000
)
cursor = conn.cursor()

# Create Database
cursor.execute('''CREATE DATABASE IF NOT EXISTS cricket_data''')
cursor.execute('''USE cricket_data''')


In [12]:
# Match Summary Tables
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS odi_match_summary (
    match_id VARCHAR(100) PRIMARY KEY,
    match_format VARCHAR(20),
    date DATE,
    city VARCHAR(100),
    venue VARCHAR(200),
    season VARCHAR(10),
    gender VARCHAR(10),
    team_1 VARCHAR(100),
    team_2 VARCHAR(100),
    toss_winner VARCHAR(100),
    toss_decision VARCHAR(20),
    winner VARCHAR(100),
    win_by_runs INT,
    win_by_wickets INT,
    overs INT,
    balls_per_over INT,
    player_of_match VARCHAR(100)
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS t20_match_summary (
    match_id VARCHAR(100) PRIMARY KEY,
    match_format VARCHAR(20),
    date DATE,
    city VARCHAR(100),
    venue VARCHAR(200),
    season VARCHAR(10),
    gender VARCHAR(10),
    team_1 VARCHAR(100),
    team_2 VARCHAR(100),
    toss_winner VARCHAR(100),
    toss_decision VARCHAR(20),
    winner VARCHAR(100),
    win_by_runs INT,
    win_by_wickets INT,
    overs INT,
    balls_per_over INT,
    player_of_match VARCHAR(100),
    target_runs INT, 
    target_overs FLOAT,
    powerplays VARCHAR(500) 
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS ipl_match_summary (
    match_id VARCHAR(100) PRIMARY KEY,
    match_format VARCHAR(20),
    date DATE,
    city VARCHAR(100),
    venue VARCHAR(200),
    season VARCHAR(10),
    gender VARCHAR(10),
    team_1 VARCHAR(100),
    team_2 VARCHAR(100),
    toss_winner VARCHAR(100),
    toss_decision VARCHAR(20),
    winner VARCHAR(100),
    win_by_runs INT,
    win_by_wickets INT,
    overs INT,
    balls_per_over INT,
    player_of_match VARCHAR(100),
    target_runs INT, 
    target_overs FLOAT,
    powerplays VARCHAR(500)
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS test_match_summary (
    match_id VARCHAR(100) PRIMARY KEY,
    match_format VARCHAR(20),
    start_date DATE,
    end_date DATE,
    city VARCHAR(100),
    venue VARCHAR(200),
    gender VARCHAR(10),
    season VARCHAR(10),
    team_1 VARCHAR(100),
    team_2 VARCHAR(100),
    toss_winner VARCHAR(100),
    toss_decision VARCHAR(20),
    winner VARCHAR(100),
    win_by_runs INT,
    win_by_wickets INT,
    balls_per_over INT,
    player_of_match VARCHAR(100)
);
""")

# --- Ball-by-ball Table Structures ---
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS odi_ball_by_ball (
    match_id VARCHAR(100),
    inning_team VARCHAR(100),
    over_number FLOAT,
    batter VARCHAR(100),
    non_striker VARCHAR(100),
    bowler VARCHAR(100),
    legbyes INT,
    wides INT,
    no_balls INT,
    byes INT,
    runs_batter INT,
    runs_extras INT,
    runs_total INT,
    wicket_kind VARCHAR(50),
    player_out VARCHAR(100)
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS test_ball_by_ball (
    match_id VARCHAR(100),
    inning_team VARCHAR(100),
    over_number FLOAT,
    batter VARCHAR(100),
    non_striker VARCHAR(100),
    bowler VARCHAR(100),
    legbyes INT,
    wides INT,
    no_balls INT,
    byes INT,
    runs_batter INT,
    runs_extras INT,
    runs_total INT,
    wicket_kind VARCHAR(50),
    player_out VARCHAR(100)
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS t20_ball_by_ball (
    match_id VARCHAR(100),
    inning_team VARCHAR(100),
    over_number FLOAT,
    batter VARCHAR(100),
    non_striker VARCHAR(100),
    bowler VARCHAR(100),
    legbyes INT,
    wides INT,
    no_balls INT,
    byes INT,
    runs_batter INT,
    runs_extras INT,
    runs_total INT,
    wicket_kind VARCHAR(50),
    player_out VARCHAR(100)
);
""")

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS ipl_ball_by_ball (
    match_id VARCHAR(100),
    inning_team VARCHAR(100),
    over_number FLOAT,
    batter VARCHAR(100),
    non_striker VARCHAR(100),
    bowler VARCHAR(100),
    legbyes INT,
    wides INT,
    no_balls INT,
    byes INT,
    runs_batter INT,
    runs_extras INT,
    runs_total INT,
    wicket_kind VARCHAR(50),
    player_out VARCHAR(100)
    
);
""")

# Player Registry Table
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS player_registry (
    match_id VARCHAR(100),
    player_name VARCHAR(100),
    player_id VARCHAR(100),
    PRIMARY KEY (match_id, player_name)
);
""")

conn.commit()
print("✅ All tables created successfully.")



✅ All tables created successfully.


In [13]:
# Now Upload CSV Data in Batches
def insert_batchwise(df, table_name, batch_size=500):
    if df.empty:
        print(f"Skipping insertion for {table_name}: DataFrame is empty.")
        return

    columns = ", ".join(df.columns)
    # Ensure placeholders match the number of columns in the DataFrame
    placeholders = ", ".join(["%s"] * len(df.columns))
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    print(f"Starting batch insertion into {table_name}...")
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size].values.tolist()
        try:
            cursor.executemany(query, batch)
            conn.commit()
            print(f"Inserted batch {i//batch_size + 1}/{len(df)//batch_size + (1 if len(df)%batch_size > 0 else 0)} ({len(batch)} rows) into {table_name}")
        except mysql.connector.Error as err:
            print(f"Error inserting batch into {table_name} (batch {i//batch_size + 1}): {err}")
            conn.rollback() # Rollback on error to prevent partial inserts


In [14]:
# Map CSV files to their corresponding table names and desired columns
csv_to_table_mapping = {
    "odi_match_summary.csv": {"table": "odi_match_summary", "cols": COMMON_SUMMARY_COLS, "date_cols": ["date"]},
    "t20_match_summary.csv": {"table": "t20_match_summary", "cols": T20_IPL_SUMMARY_COLS, "date_cols": ["date"]},
    "ipl_match_summary.csv": {"table": "ipl_match_summary", "cols": T20_IPL_SUMMARY_COLS, "date_cols": ["date"]},
    "test_match_summary.csv": {"table": "test_match_summary", "cols": TEST_SUMMARY_COLS, "date_cols": ["start_date", "end_date"]},
    "odi_ball_by_ball.csv": {"table": "odi_ball_by_ball", "cols": COMMON_BALL_BY_BALL_COLS, "date_cols": []},
    "t20_ball_by_ball.csv": {"table": "t20_ball_by_ball", "cols": T20_IPL_BALL_BY_BALL_COLS, "date_cols": []},
    "ipl_ball_by_ball.csv": {"table": "ipl_ball_by_ball", "cols": T20_IPL_BALL_BY_BALL_COLS, "date_cols": []},
    "test_ball_by_ball.csv": {"table": "test_ball_by_ball", "cols": COMMON_BALL_BY_BALL_COLS, "date_cols": []},
    "player_registry.csv": {"table": "player_registry", "cols": PLAYER_REGISTRY_COLS, "date_cols": []}
}

# Upload all CSVs
for csv_file, details in csv_to_table_mapping.items():
    table_name = details["table"]
    desired_columns = details["cols"]
    date_columns = details["date_cols"]

    try:
        if not os.path.exists(csv_file):
            print(f"CSV file not found: {csv_file}. Skipping.")
            continue

        df = pd.read_csv(csv_file)
        
        # Apply the comprehensive cleanup and column selection
        df = clean_and_select_dataframe(df, desired_columns=desired_columns, df_name=csv_file)

        # Handle date column conversion (specific to summary tables)
        for date_col in date_columns:
            if date_col in df.columns:
                # Convert 'NA' string back to NaN for datetime conversion
                df[date_col] = df[date_col].replace("NA", np.nan)
                df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
                # Convert NaT (Not a Time) back to None or specific placeholder for database
                # MySQL connector often handles None as NULL, which is appropriate for DATE columns
                df[date_col] = df[date_col].replace({pd.NaT: None}) 
        
        insert_batchwise(df, table_name, batch_size=500)

    except Exception as e:
        print(f"Failed to process and insert {csv_file} into {table_name}: {e}")

cursor.close()
conn.close()
print("\n All available CSV data processed and uploaded to TiDB.")


Starting batch insertion into odi_match_summary...
Inserted batch 1/6 (500 rows) into odi_match_summary
Inserted batch 2/6 (500 rows) into odi_match_summary
Inserted batch 3/6 (500 rows) into odi_match_summary
Inserted batch 4/6 (500 rows) into odi_match_summary
Inserted batch 5/6 (500 rows) into odi_match_summary
Inserted batch 6/6 (486 rows) into odi_match_summary
Starting batch insertion into t20_match_summary...
Inserted batch 1/9 (500 rows) into t20_match_summary
Inserted batch 2/9 (500 rows) into t20_match_summary
Inserted batch 3/9 (500 rows) into t20_match_summary
Inserted batch 4/9 (500 rows) into t20_match_summary
Inserted batch 5/9 (500 rows) into t20_match_summary
Inserted batch 6/9 (500 rows) into t20_match_summary
Inserted batch 7/9 (500 rows) into t20_match_summary
Inserted batch 8/9 (500 rows) into t20_match_summary
Inserted batch 9/9 (468 rows) into t20_match_summary
Starting batch insertion into ipl_match_summary...
Inserted batch 1/3 (500 rows) into ipl_match_summary